In [7]:
import logging
import json
import random
import time
import numpy as np
import pickle
from gensim.models import doc2vec
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
from crashsimilarity.downloader import SocorroDownloader, Downloader
from crashsimilarity import utils
from itertools import islice

In [4]:
def corpus_generator(file_name):
    with open(file_name, 'r') as f:
        for i, line in enumerate(f):
            j = json.loads(line)
            stack_trace = j[0]
            yield doc2vec.TaggedDocument(stack_trace, [i])

In [6]:
gen = corpus_generator('data/new_clean_compressed.json')
corpus = list(gen)

In [15]:
c_10 = Counter()
c_15 = Counter()
c_20 = Counter()
c_25 = Counter()
for i, data in enumerate(corpus):
    if i % 100000 == 0:
        print(i)
    c_10[tuple(data[0][-10:])] += 1
    c_15[tuple(data[0][-15:])] += 1
    c_20[tuple(data[0][-20:])] += 1
    c_25[tuple(data[0][-25:])] += 1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000


In [16]:
c_25.most_common(15)

[(('211',
   '212',
   '213',
   '214',
   '215',
   '216',
   '217',
   '218',
   '219',
   '15',
   '68',
   '69',
   '70',
   '23',
   '24',
   '72',
   '73',
   '220',
   '221',
   '125',
   '126',
   '156',
   '28',
   '29',
   '222'),
  2133),
 (('210',
   '211',
   '212',
   '213',
   '214',
   '215',
   '216',
   '217',
   '218',
   '219',
   '15',
   '68',
   '70',
   '23',
   '24',
   '72',
   '73',
   '220',
   '221',
   '125',
   '126',
   '156',
   '28',
   '29',
   '222'),
  1984),
 (('201',
   '202',
   '430',
   '431',
   '214',
   '215',
   '216',
   '217',
   '218',
   '219',
   '15',
   '68',
   '70',
   '23',
   '24',
   '72',
   '73',
   '220',
   '221',
   '125',
   '126',
   '156',
   '28',
   '29',
   '222'),
  1206),
 (('397',
   '63',
   '64',
   '65',
   '66',
   '67',
   '68',
   '70',
   '71',
   '23',
   '24',
   '72',
   '73',
   '74',
   '71',
   '23',
   '24',
   '25',
   '26',
   '27',
   '28',
   '29',
   '30',
   '31',
   '32'),
  1027),
 (('906',
  